In [59]:

from src.data.dataset_reader import DatasetReader
from pathlib import Path
%load_ext autoreload
%autoreload 2
# ignore warnings
import warnings
warnings.filterwarnings('ignore')
data_folder = Path("../data/raw").resolve()

amazon = DatasetReader(data_folder).get_data_csv('Amazon Sale Report.csv')
#int_sales = DatasetReader(data_folder).get_data_csv('International sale Report.csv')
#p_l = DatasetReader(data_folder).get_data_csv('P LMarch2021.csv')
#may_2022 = DatasetReader(data_folder).get_data_csv('May-2022.csv')
#sale_report = DatasetReader(data_folder).get_data_csv('Sale Report.csv')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
import pandas as pd
pd.set_option('display.max_columns', None)
amazon.head(3)
#amazon.query('SKU == "SET389-KR-NP-S"').head(1)
                                             


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,NaN,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,1,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,1,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN


In [61]:
from src.data.feature_engineering import FeatureEngineeringProcess
fep = FeatureEngineeringProcess()


In [3]:
# Create a new feature called sales that is the  the count of "Amount" grouped by "SKU" in the amazon dataset
#amazon['sales'] = amazon.groupby('SKU')['Amount'].transform('count')
#amazon.drop(columns=[ 'Order ID', 'ASIN', 'Unnamed: 22', 'index'], inplace=True
amazon['Date'] = pd.to_datetime(amazon['Date'])
amazon['ship-postal-code'] = amazon['ship-postal-code'].astype('object')
amazon_beta =  amazon[['Order ID','Date', 'SKU', 'Qty', 'Amount']]

In [4]:
from src.data.eda import Eda
eda = Eda(    
)

eda.missing_values_table(amazon_beta)

The selected dataframe has 5 columns and 1 columns with missing values.


,Missing Values,% of Total Values
Amount,7795,6.0


In [5]:
amazon_beta.rename(columns={'Amount': 'price'}, inplace=True)

In [6]:
amazon_beta.head()
# Create a column called "sales" that is the count of the "Order ID" grouped by "SKU" and "Date" in the amazon_beta dataset
amazon_beta['sales'] = amazon_beta.groupby(['SKU', 'Date'])['Order ID'].transform('count')


In [ ]:
#amazon_beta.query('SKU == "SET389-KR-NP-S"').sort_values(by=['Date'], ascending=True).head()

In [ ]:
# show duplicates in amazon_beta 
#amazon_beta[amazon_beta.duplicated(keep=False)].sort_values(by='SKU', ascending=True).head(10)
# drop duplicates in amazon_beta
#amazon_beta.drop_duplicates(inplace=True)

In [ ]:
eda.missing_values_table(amazon_beta)

In [7]:
amazon_beta.sort_values(by=['SKU', 'Date'], ascending=True, inplace=True)

In [8]:
from src.data.feature_engineering import FeatureEngineeringProcess
fep = FeatureEngineeringProcess()

In [9]:
amazon_beta_fe1 = fep.price_sales_correlation_features_updated(amazon_beta, 15, [1, 2], [1,2],)
#amazon_beta_fe1.head(3)

In [ ]:
#print this instruction: amazon_beta_fe1.query('SKU == "SET389-KR-NP-S"') without index
#amazon_beta_fe1.query('SKU == "SET389-KR-NP-S"').head(3)

In [10]:
amazon_beta_fe2 = fep.normalize_features(amazon_beta_fe1, 8, 15)

In [ ]:
#amazon_beta_fe2.query('SKU == "SET389-KR-NP-S"').T

In [11]:
amazon_beta_fe3 = fep.filter_stability_periods(amazon_beta_fe2, 7, 0.04)

In [ ]:
# print without index amazon_beta_fe3.query('SKU == "SET389-KR-NP-S"') without index column


#amazon_beta_fe3[['Order ID', 'Date', 'SKU', 'price', 'sales','avg_price_last_7_days', 'price_variation']].query('SKU == "SET389-KR-NP-S"')


# NOTE: Still seeing repeated values in the "price" column. Either low the threshold or increase removal_percentage


In [ ]:

eda.missing_values_table(amazon_beta_fe3)

In [ ]:
amazon_beta_fe3.head()

In [12]:
# drop rows where price is null
amazon_beta_train = amazon_beta_fe3.dropna(subset=['price'])

In [13]:
amazon_beta_train.fillna(0, inplace=True)

In [53]:
amazon_beta_train1= fep.datetime_transform(amazon_beta_train, 'Date',['day_of_the_year', 'month', 'day_of_the_week'])

In [51]:
# Drop Order Id, price_variation, insufficient_data
amazon_beta_train1.drop(columns=['Order ID', 'price_variation', 'insufficient_data'], inplace=True)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/qd/2qwtnqg57r915dp6p3dq762c0000gn/T/ipykernel_10141/2553612276.py:2 in <cell line:  │
│ 2>                                                                                               │
│                                                                                                  │
│                                                                                                  │
│ /Users/servandodavidtorresgarcia/miniconda3/envs/relu/lib/python3.8/site-packages/pandas/util/_d │
│ ecorators.py:311 in wrapper                                                                      │
│                                                                                                  │
│   310 │   │   │   │   )                                                                          │
│ ❱ 311 │   │   │   return func(*args, **kwargs)                                                   │
│   312                                                                                            │
│                                                                                                  │
│ /Users/servandodavidtorresgarcia/miniconda3/envs/relu/lib/python3.8/site-packages/pandas/core/fr │
│ ame.py:4954 in drop                                                                              │
│                                                                                                  │
│    4953 │   │   """                                                                              │
│ ❱  4954 │   │   return super().drop(                                                             │
│    4955 │   │   │   labels=labels,                                                               │
│                                                                                                  │
│ /Users/servandodavidtorresgarcia/miniconda3/envs/relu/lib/python3.8/site-packages/pandas/core/ge │
│ neric.py:4267 in drop                                                                            │
│                                                                                                  │
│    4266 │   │   │   if labels is not None:                                                       │
│ ❱  4267 │   │   │   │   obj = obj._drop_axis(labels, axis, level=level, errors=errors)           │
│    4268                                                                                          │
│                                                                                                  │
│ /Users/servandodavidtorresgarcia/miniconda3/envs/relu/lib/python3.8/site-packages/pandas/core/ge │
│ neric.py:4311 in _drop_axis                                                                      │
│                                                                                                  │
│    4310 │   │   │   else:                                                                        │
│ ❱  4311 │   │   │   │   new_axis = axis.drop(labels, errors=errors)                              │
│    4312 │   │   │   indexer = axis.get_indexer(new_axis)                                         │
│                                                                                                  │
│ /Users/servandodavidtorresgarcia/miniconda3/envs/relu/lib/python3.8/site-packages/pandas/core/in │
│ dexes/base.py:6644 in drop                                                                       │
│                                                                                                  │
│   6643 │   │   │   if errors != "ignore":                                                        │
│ ❱ 6644 │   │   │   │   raise KeyError(f"{list(labels[mask])} not found in axis")                 │
│   6645 │   │   │   indexer = indexer[~mask]                                                      │
╰────────────────────────────────────────────────────────────

In [55]:
#amazon_beta_train1.head()
amazon_beta_train1.drop(columns=['Date_season'], inplace=True)

In [56]:
from src.data.data_splitter import DataSplitter

splitter = DataSplitter(amazon_beta_train1, train_months=1, test_weeks=2)
train, test = splitter.split_data()

Min Date: 2022-03-31 00:00:00, Max Date: 2022-06-29 00:00:00, Total Months: 3
Train data types:
Date                        datetime64[ns]
SKU                                 object
Qty                                  int64
price                              float64
sales                                int64
f_corr_1_1                         float64
f_corr_1_2                         float64
f_corr_2_1                         float64
f_corr_2_2                         float64
normalized_avg_price               float64
normalized_avg_sales               float64
normalized_log_avg_price           float64
normalized_log_avg_sales           float64
normalized_std_price               float64
normalized_std_sales               float64
avg_price_last_7_days              float64
Date_month                           int64
Date_day_of_the_year                 int64
dtype: object

Test data types:
Date                        datetime64[ns]
SKU                                 object
Qty         

In [18]:
train.shape, test.shape
# print min and max dates of train and test
train['Date'].min(), train['Date'].max(), test['Date'].min(), test['Date'].max()
# Min Date: 2022-03-31 00:00:00, Max Date: 2022-06-29 00:00:00, Total Months: 3

(Timestamp('2022-03-31 00:00:00'),
 Timestamp('2022-05-29 00:00:00'),
 Timestamp('2022-05-30 00:00:00'),
 Timestamp('2022-06-29 00:00:00'))

In [57]:
# Drop Date column
train.drop(columns=['Date'], inplace=True)
test.drop(columns=['Date'], inplace=True)
# one hot encode Date_season
#train = pd.get_dummies(train, columns=['Date_season'])
#test = pd.get_dummies(test, columns=['Date_season'])

In [ ]:
# Clean memory usage and delete dataframes that are not needed
#import gc
#del amazon, amazon_beta, amazon_beta_fe1, amazon_beta_fe2, amazon_beta_fe3, amazon_beta_train, amazon_beta_train1
#gc.collect()



In [58]:
# dump as pickel files train and test dataframes in the data/processed folder
train.to_pickle('../data/processed/train.pkl')
test.to_pickle('../data/processed/test.pkl')

In [38]:
# load train and test pickle from the data/processed folder
train = pd.read_pickle('../data/processed/train.pkl')

In [40]:
# load test pickle from the data/processed folder
test = pd.read_pickle('../data/processed/test.pkl')

In [42]:
train.columns

Index(['SKU', 'Qty', 'price', 'sales', 'f_corr_1_1', 'f_corr_1_2',
       'f_corr_2_1', 'f_corr_2_2', 'normalized_avg_price',
       'normalized_avg_sales', 'normalized_log_avg_price',
       'normalized_log_avg_sales', 'normalized_std_price',
       'normalized_std_sales', 'avg_price_last_7_days', 'Date_month',
       'Date_day_of_the_year', 'Date_season_Spring'],
      dtype='object')

In [ ]:
import psutil

# Get memory usage in bytes
memory_usage = psutil.virtual_memory()
total_memory = memory_usage.total
available_memory = memory_usage.available
used_memory = memory_usage.used
percent_memory_used = memory_usage.percent

# Convert memory values to human-readable format
total_memory_readable = psutil._common.bytes2human(total_memory)
available_memory_readable = psutil._common.bytes2human(available_memory)
used_memory_readable = psutil._common.bytes2human(used_memory)

# Print memory usage information
print(f"Total Memory: {total_memory_readable}")
print(f"Available Memory: {available_memory_readable}")
print(f"Used Memory: {used_memory_readable}")
print(f"Percent Memory Used: {percent_memory_used}%")


In [ ]:
may = 467
june = 467
july = 497
august = 498
september = 515
october = 500
november = 515
december = 523

# get total sum of sales for each month

total_sales = may + june + july + august + september + october + november + december


In [ ]:
import modal

stub = modal.Stub("example-get-started")
@stub.function()
def f():
    return "hello"

with stub.run():
    assert "hello" == f.call()  # remote call works ✔️
    f()  # raises error in notebook ⚠️

In [36]:
#check version of pandas
pd.__version__

'1.4.2'

In [37]:
train.shape, test.shape

((43121, 18), (20316, 19))

In [45]:
test.columns

Index(['SKU', 'Qty', 'price', 'sales', 'f_corr_1_1', 'f_corr_1_2',
       'f_corr_2_1', 'f_corr_2_2', 'normalized_avg_price',
       'normalized_avg_sales', 'normalized_log_avg_price',
       'normalized_log_avg_sales', 'normalized_std_price',
       'normalized_std_sales', 'avg_price_last_7_days', 'Date_month',
       'Date_day_of_the_year', 'Date_season_Spring', 'Date_season_Summer'],
      dtype='object')

In [49]:
train.columns

Index(['SKU', 'Qty', 'price', 'sales', 'f_corr_1_1', 'f_corr_1_2',
       'f_corr_2_1', 'f_corr_2_2', 'normalized_avg_price',
       'normalized_avg_sales', 'normalized_log_avg_price',
       'normalized_log_avg_sales', 'normalized_std_price',
       'normalized_std_sales', 'avg_price_last_7_days', 'Date_month',
       'Date_day_of_the_year', 'Date_season_Spring'],
      dtype='object')

In [48]:
test.shape

(20316, 19)